In [1]:
import torch

In [2]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

In [10]:
# 판다스로 훈련셋과 테스트셋 데이터 로드
train = pd.read_csv("dataset/article_LT_Labeling(20200505-20200731).csv", sep=',')
test = pd.read_csv("dataset/article_SS_Labeling(20200505-20200731).csv", sep=',')

In [11]:
del train['id']
del train['team']
del train['date']
del train['image']
del train['positive']
del train['negative']

del test['id']
del test['team']
del test['date']
del test['image']
del test['positive']
del test['negative']

In [12]:
print(train.shape)
print(test.shape)

(1163, 2)
(948, 2)


In [13]:
# 훈련셋의 앞부분 출력
train.head(10)

,contents,positive_or_negative
0,"""장타력 주목"" 서용빈 해설이 찍은 마차도, 개막전부터 미친 존재감 [★현장][스타...",1
1,"'마차도 역전 스리런' 롯데, KT에 7:2 역전승(종합)마차도. (C)롯데 ...",1
2,"'4타점 활약' 마차도 ""내가 '최고'라는 마음으로""KBO리그 데뷔전서 2안타 1홈...",1
3,"'데뷔전 승리' 허문회 감독, ""이제 시작이죠"" [수원 톡톡][OSEN=수원, 조은...",1
4,"롯데, 4년 만에 짜릿한 개막전 축포2020 KBO 리그 개막전이 5일 각 구장에서...",1
5,‘첫 승’ 허문회 롯데 감독 “이제부터 시작이다”[스포츠월드=수원 이혜진 기자] “...,1
6,"'4타점' 롯데 마차도의 좋은 마인드 ""내가 최고다""[마이데일리 = 수원 김진성 기...",1
7,"'4타점 원맨쇼' 마차도 ""타격에서도 항상 최고라는 생각 갖는다"" [수원 S트리밍]...",1
8,"'데뷔전 승리' 허문회 롯데 감독 ""이제 시작""롯데, 개막전서 KT 7-2 제압 [...",1
9,"'홈런&4타점 데뷔전' 마차도, ""다같이 노력한 덕분에 좋은 결과"" [수원 톡톡][...",1


In [15]:
# 리뷰 문장 추출
sentences = train['contents']
sentences[:10]

0    "장타력 주목" 서용빈 해설이 찍은 마차도, 개막전부터 미친 존재감 [★현장][스타...
1    '마차도 역전 스리런' 롯데, KT에 7:2 역전승(종합)마차도. (C)롯데    ...
2    '4타점 활약' 마차도 "내가 '최고'라는 마음으로"KBO리그 데뷔전서 2안타 1홈...
3    '데뷔전 승리' 허문회 감독, "이제 시작이죠" [수원 톡톡][OSEN=수원, 조은...
4    롯데, 4년 만에 짜릿한 개막전 축포2020 KBO 리그 개막전이 5일 각 구장에서...
5    ‘첫 승’ 허문회 롯데 감독 “이제부터 시작이다”[스포츠월드=수원 이혜진 기자] “...
6    '4타점' 롯데 마차도의 좋은 마인드 "내가 최고다"[마이데일리 = 수원 김진성 기...
7    '4타점 원맨쇼' 마차도 "타격에서도 항상 최고라는 생각 갖는다" [수원 S트리밍]...
8    '데뷔전 승리' 허문회 롯데 감독 "이제 시작"롯데, 개막전서 KT 7-2 제압 [...
9    '홈런&4타점 데뷔전' 마차도, "다같이 노력한 덕분에 좋은 결과" [수원 톡톡][...
Name: contents, dtype: object

In [16]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] "장타력 주목" 서용빈 해설이 찍은 마차도, 개막전부터 미친 존재감 [★현장][스타뉴스 수원=한동훈 기자] 롯데 마차도가 5일 수원 KT전 승리 후 수훈선수 인터뷰를 하고 있다. /사진=한동훈 기자 롯데 자이언츠 새 외국인타자 딕슨 마차도(28)가 개막전부터 미친 존재감을 발휘했다.  마차도는 5일 수원 KT위즈파크에서 열린 2020 KBO리그 개막전 KT 위즈전에 7번 타자 겸 유격수로 선발 출전, 4타수 2안타 1홈런 4타점을 기록했다. 롯데는 7-2로 역전승을 거뒀다.  타격코치 출신 서용빈 해설위원이 시즌 전 "마차도의 장타력을 주목해야 한다"고 내다봤는데 예상이 개막전부터 적중한 셈이다.  서용빈 위원은 개막에 앞서 실시한 스타뉴스 설문조사에서 마차도를 눈여겨 봐야 한다고 분석했다. 두산 페르난데스나 SK 로맥, KT 로하스 등 검증된 선수가 아니라 마차도를 콕 찍었다. 서 위원은 최근 마차도의 장타력이 눈에 띄게 발전했다고 짚었다.  서 위원은 "마차도가 마이너리그에서 2015년부터 2018년까지 홈런을 11개를 쳤다. 그런데 2019년에만 17개를 때렸다. 마차도는 올해 28세다. 나이를 보면 파워 향상이 아직도 진행 중이라는 뜻이다. 수비도 상당히 좋더라"고 높이 평가했다.  마차도는 4월 21일부터 진행된 연습경기서 부진했다. 6경기 16타수 2안타 타율 0.125에 그쳤다.  하지만 5일, 막상 본 게임에 돌입하자 매서운 파괴력을 자랑했다. 마차도는 승부처에서 번뜩이는 해결사 능력을 뽐냈다. 0-1로 끌려가던 5회초, 무사 2루서 KT 데스파이네를 상대로 좌중간 적시타를 때려 동점을 만들었다.  1-2로 뒤진 7회초에는 역전 3점 홈런을 폭발시켰다. 1-1로 맞선 6회말, 롯데가 강백호에게 솔로 홈런을 맞아 흐름이 넘어가려던 차였다. 마차도는 바로 다음 공격인 7회초, 분위기를 뒤엎었다. 무사 1, 2루서 KT 필승조 김재윤을 무너뜨리는 대형 아치를 뿜었다. 2볼 1스트라이크의 유리한 카운트서 높은 코스의 패스트볼을 놓치지 않고

In [18]:
# 라벨 추출
labels = train['positive_or_negative'].values
labels

array([1, 1, 1, ..., 0, 0, 0], dtype=int64)

In [19]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS] "장타력 주목" 서용빈 해설이 찍은 마차도, 개막전부터 미친 존재감 [★현장][스타뉴스 수원=한동훈 기자] 롯데 마차도가 5일 수원 KT전 승리 후 수훈선수 인터뷰를 하고 있다. /사진=한동훈 기자 롯데 자이언츠 새 외국인타자 딕슨 마차도(28)가 개막전부터 미친 존재감을 발휘했다.  마차도는 5일 수원 KT위즈파크에서 열린 2020 KBO리그 개막전 KT 위즈전에 7번 타자 겸 유격수로 선발 출전, 4타수 2안타 1홈런 4타점을 기록했다. 롯데는 7-2로 역전승을 거뒀다.  타격코치 출신 서용빈 해설위원이 시즌 전 "마차도의 장타력을 주목해야 한다"고 내다봤는데 예상이 개막전부터 적중한 셈이다.  서용빈 위원은 개막에 앞서 실시한 스타뉴스 설문조사에서 마차도를 눈여겨 봐야 한다고 분석했다. 두산 페르난데스나 SK 로맥, KT 로하스 등 검증된 선수가 아니라 마차도를 콕 찍었다. 서 위원은 최근 마차도의 장타력이 눈에 띄게 발전했다고 짚었다.  서 위원은 "마차도가 마이너리그에서 2015년부터 2018년까지 홈런을 11개를 쳤다. 그런데 2019년에만 17개를 때렸다. 마차도는 올해 28세다. 나이를 보면 파워 향상이 아직도 진행 중이라는 뜻이다. 수비도 상당히 좋더라"고 높이 평가했다.  마차도는 4월 21일부터 진행된 연습경기서 부진했다. 6경기 16타수 2안타 타율 0.125에 그쳤다.  하지만 5일, 막상 본 게임에 돌입하자 매서운 파괴력을 자랑했다. 마차도는 승부처에서 번뜩이는 해결사 능력을 뽐냈다. 0-1로 끌려가던 5회초, 무사 2루서 KT 데스파이네를 상대로 좌중간 적시타를 때려 동점을 만들었다.  1-2로 뒤진 7회초에는 역전 3점 홈런을 폭발시켰다. 1-1로 맞선 6회말, 롯데가 강백호에게 솔로 홈런을 맞아 흐름이 넘어가려던 차였다. 마차도는 바로 다음 공격인 7회초, 분위기를 뒤엎었다. 무사 1, 2루서 KT 필승조 김재윤을 무너뜨리는 대형 아치를 뿜었다. 2볼 1스트라이크의 유리한 카운트서 높은 코스의 패스트볼을 놓치지 않고 강

In [20]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,    107,   9657,  22695,  28143,   9689,  68055,    107,
         9425,  24974, 102360,   9960,  31928,  10739,   9728,  10892,
         9246,  23466,  12092,    117,   8857, 118907,  16617,  17655,
         9309,  55358,   9680,  36210, 105197,    164,   1861,  30842,
        13890,    166,    164,   9477,  22695,  90947,   9460,  14279,
          134,   9954,  18778,  75965,  60886,    166,   9208,  28911,
         9246,  23466,  68516,  44015,   9460,  14279,    148,  11090,
        16617,   9484,  12692,  10003,   9460,  75965,  18471,  15891,
         9640,  21876, 118999,  11513,  32487,  11506,    119,    120,
         9405,  18623,    134,   9954,  18778,  75965,  60886,   9208,
        28911,   9651,  10739,  48036,  60479,   9415,   9597,  20479,
        12030,  22695,  13764,   9123,  50472,   9246,  23466,  12092,
          113,  10348,    114,   8843,   8857, 118907,  16617,  17655,
         9309,  55358,   9680,  36210, 105197,  10622,   9323, 119455,
      

In [21]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [22]:
# 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=2018, 
                                                                                    test_size=0.1)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=2018, 
                                                       test_size=0.1)

# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)				

print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
print(validation_inputs[0])
print(validation_labels[0])
print(validation_masks[0])

tensor([   101,    164,   9460,  14279,   9812, 118929,  15184,    166,    112,
          9566,  16617,   9477,  12692,  56710,    112,   9450,  16985, 119041,
         10739,   9324, 119469,   9110,  10739,  21711,  12310,    117,   9208,
         28911,   9968,  25934,  14863,  81571,  10739,   8865, 118736,   9251,
         10892,    136,  46659,   9460,  14279,    148,  11090,  19855,  24891,
         46150,  20308,  11489,    148,  11090,   9619,  24891,  12638,   9208,
         28911,   9651,  10739,  48036,  60479,  10459,  86015,  11287,   9569,
         45554,    119,    128,  14863,  57030,    123,  12945,    122,    117,
           123,  35866,  11489,   9450,  16985, 119041,  10739,   9566,  16617,
           124,  34907,   9989,  56710,  10622,   8985,  45554,    119,   9075,
         16985, 119170,  11489,   9095,  38688,  72482,   9952,  10739,  46150,
         98789,  11513,   8982, 118751,  11664,  13767,   9450,  16985, 119041,
           119,   9460,  14279,    134, 

In [23]:
# 배치 사이즈
batch_size = 8

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [25]:
# 리뷰 문장 추출
sentences = test['contents']
sentences[:10]

0    “대구 힘내세요” “기억할게요”…희망 주는 ‘야구’가 돌아왔다ㆍ무관중 개막 삼성 -...
1    올해도 졌다…삼성 라팍 홈 개막전 ‘무승 5패’ [2020 프로야구]매경닷컴 MK스...
2    "대구시민 덕분에 개막"…코로나 딛고 프로야구 '플레이볼'무관중에 ESPN 중계까지...
3    삼성, 아쉬운 출발…개막전 NC에 0대4로 져백정현 선발 출격, NC킬러 무색 5일...
4    삼성라이온즈 허삼영 감독 “야구로 희망과 기쁨 드리고 싶습니다”선수들과 팀이 가고자...
5    “대구에서 야구하게 해줘 감사합니다” 삼성의 특별한 개막전이성구 대구시 의사협회장이...
6    '6이닝 완벽투' 루친스키, NC의 산뜻한 출발 이끌다 [오!쎈人][OSEN=대구,...
7    'NC 천적' 백정현, 피홈런 3개 허용…NC전 통산 2패 위기[일간스포츠 배중현]...
8    이성구 대구시 의사협회장, "코로나19 극복은 성숙한 시민의식 덕분" [오!쎈 인터...
9    이원석, "팬들 응원 속에 신나게 야구하는 날 왔으면" [대구 톡톡][OSEN=대구...
Name: contents, dtype: object

In [26]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] “대구 힘내세요” “기억할게요”…희망 주는 ‘야구’가 돌아왔다ㆍ무관중 개막 삼성 - NC전 관중석엔 “함께 극복을” 현수막 ㆍ코로나19 의료진 응원 영상에 선수들도 ‘덕분에’ 손동작 ㆍ각 구장 깜짝 시구·재치있는 플래카드 ‘야구의 계절’ 환영  관중은 없어도 오늘은 좋은 날 SK 치어리더들이 5일 인천 SK행복드림구장에서 열린 한화와의 KBO리그 개막전에서 온라인으로 생중계되는 응원을 펼치고 있다. 인천 | 이석우 기자   5월5일. 어린이들이 가득 차야 할 대구 삼성라이온즈파크 외야 관중석에는 ‘힘내라 대한민국 그동안 수고하셨습니다. 마지막까지 힘을 모아 함께 극복합시다!’라는 대형 현수막이 걸렸다. 연고팀 삼성이 코로나19로 어려움을 겪은 대구·경북 주민들을 향해 보내는 기원 메시지였다.  지난 2월 일본 오키나와에서 스프링캠프를 치를 때까지만 해도 삼성은 야구를 꿈꾸기 어려웠다. 대구·경북 지역은 코로나19 확진자가 가장 많이 나온 곳이었다. 삼성은 일본의 입국 제한 조치 여파로 3월7일 급히 짐을 쌌다. 귀국 뒤에도 철저한 규제 속에서 고립된 채 조용히 시즌을 준비했다.  수많은 이들의 노력 끝에 코로나19 사태가 진정세에 접어들면서 올 것 같지 않았던 개막이 다가왔다. 삼성 주장 박해민은 지난 3일 미디어데이에서 “대구에서 야구할 수 있게 해주셔서 감사하다”고 했다. 이날 개막전에서도 ‘덕분에’라는 감사와 ‘힘내라’라는 의지가 한데 어우러졌다. 허삼영 삼성 감독은 “대구·경북 지역에 야구가 희망을 전달할 수 있도록 팀이 똘똘 뭉치겠다”고 말했다.  삼성은 개막전 시구자로 최일선에서 코로나19와 싸우고 전국 의료진의 지원을 끌어낸 이성구 대구시의사협회장을 세웠다. 경기가 시작되기 전 코로나19 환자 진료와 치료에 헌신하는 의료진을 응원하는 캠페인인 ‘덕분에’ 챌린지와 관련된 영상이 전광판을 통해 나왔다.  팬들과 선수들이 ‘존경’과 ‘자부심’을 뜻하는 수어 동작인 왼손 위로 오른손 엄지를 드는 자세를 한 사진으로 만들어진 영상이었다. 그리

In [28]:
# 라벨 추출
labels = test['positive_or_negative'].values
labels

array([0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0,
       1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,

In [29]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS] “대구 힘내세요” “기억할게요”…희망 주는 ‘야구’가 돌아왔다ㆍ무관중 개막 삼성 - NC전 관중석엔 “함께 극복을” 현수막 ㆍ코로나19 의료진 응원 영상에 선수들도 ‘덕분에’ 손동작 ㆍ각 구장 깜짝 시구·재치있는 플래카드 ‘야구의 계절’ 환영  관중은 없어도 오늘은 좋은 날 SK 치어리더들이 5일 인천 SK행복드림구장에서 열린 한화와의 KBO리그 개막전에서 온라인으로 생중계되는 응원을 펼치고 있다. 인천 | 이석우 기자   5월5일. 어린이들이 가득 차야 할 대구 삼성라이온즈파크 외야 관중석에는 ‘힘내라 대한민국 그동안 수고하셨습니다. 마지막까지 힘을 모아 함께 극복합시다!’라는 대형 현수막이 걸렸다. 연고팀 삼성이 코로나19로 어려움을 겪은 대구·경북 주민들을 향해 보내는 기원 메시지였다.  지난 2월 일본 오키나와에서 스프링캠프를 치를 때까지만 해도 삼성은 야구를 꿈꾸기 어려웠다. 대구·경북 지역은 코로나19 확진자가 가장 많이 나온 곳이었다. 삼성은 일본의 입국 제한 조치 여파로 3월7일 급히 짐을 쌌다. 귀국 뒤에도 철저한 규제 속에서 고립된 채 조용히 시즌을 준비했다.  수많은 이들의 노력 끝에 코로나19 사태가 진정세에 접어들면서 올 것 같지 않았던 개막이 다가왔다. 삼성 주장 박해민은 지난 3일 미디어데이에서 “대구에서 야구할 수 있게 해주셔서 감사하다”고 했다. 이날 개막전에서도 ‘덕분에’라는 감사와 ‘힘내라’라는 의지가 한데 어우러졌다. 허삼영 삼성 감독은 “대구·경북 지역에 야구가 희망을 전달할 수 있도록 팀이 똘똘 뭉치겠다”고 말했다.  삼성은 개막전 시구자로 최일선에서 코로나19와 싸우고 전국 의료진의 지원을 끌어낸 이성구 대구시의사협회장을 세웠다. 경기가 시작되기 전 코로나19 환자 진료와 치료에 헌신하는 의료진을 응원하는 캠페인인 ‘덕분에’ 챌린지와 관련된 영상이 전광판을 통해 나왔다.  팬들과 선수들이 ‘존경’과 ‘자부심’을 뜻하는 수어 동작인 왼손 위로 오른손 엄지를 드는 자세를 한 사진으로 만들어진 영상이었다. 그리고 

In [30]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,    100,   9069,  17196,  10028,  31605,  24982,  48549,
          100,    100,   8932,  91837,  14843,  14153,  48549,    100,
          100,  10023,  89292,   9689,  11018,    100, 106603,    100,
         8843,   9091,  16985,  72995, 111836,  32537,  20595,  41693,
         8857, 118907,   9410,  17138,    118,  55838,  16617,   8900,
        41693,  40958,  86933,    100,  19653,   8925,  70915,  10622,
          100,   9978,  15891, 118907,   2070,  25517,  11261,  16439,
        54055,   9637,  38688,  18623,   9636,  14279,   9574,  14871,
        10530,  78930,  27023,  12092,    100,   9075, 110355,    100,
         9450,  18778,  38709,   2070,  66540,   8908,  13890,   8943,
       119236,   9485,  17196,    217,   9659,  18622,  84177,   9944,
        37388,  24206,  15001,    100, 106603,  10459,   8887,  58931,
          100,   9995,  30858,   8900,  41693,  10892,   9555,  12965,
        12092,   9580, 118762,  10892,  79633,   8985,  21275,   9779,
      

In [31]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [32]:
# 데이터를 파이토치의 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

print(test_inputs[0])
print(test_labels[0])
print(test_masks[0])

tensor([   101,    100,   9069,  17196,  10028,  31605,  24982,  48549,    100,
           100,   8932,  91837,  14843,  14153,  48549,    100,    100,  10023,
         89292,   9689,  11018,    100, 106603,    100,   8843,   9091,  16985,
         72995, 111836,  32537,  20595,  41693,   8857, 118907,   9410,  17138,
           118,  55838,  16617,   8900,  41693,  40958,  86933,    100,  19653,
          8925,  70915,  10622,    100,   9978,  15891, 118907,   2070,  25517,
         11261,  16439,  54055,   9637,  38688,  18623,   9636,  14279,   9574,
         14871,  10530,  78930,  27023,  12092,    100,   9075, 110355,    100,
          9450,  18778,  38709,   2070,  66540,   8908,  13890,   8943, 119236,
          9485,  17196,    217,   9659,  18622,  84177,   9944,  37388,  24206,
         15001,    100, 106603,  10459,   8887,  58931,    100,   9995,  30858,
          8900,  41693,  10892,   9555,  12965,  12092,   9580, 118762,  10892,
         79633,   8985,  21275,   9779, 

In [33]:
# 배치 사이즈
batch_size = 8

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [34]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [35]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: GeForce GTX 1080 Ti


In [36]:
# 분류를 위한 BERT 모델 생성
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [37]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 4

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 처음에 학습률을 조금씩 변화시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [38]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [39]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [40]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
torch.backends.cudnn.enabled = False

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
       
        # Forward 수행                
        outputs = model(torch.tensor(b_input_ids).to(device).long(), 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(torch.tensor(b_input_ids).to(device).long(), 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...


C:\Users\DeepLearning_1\anaconda3\envs\windup\lib\site-packages\ipykernel_launcher.py:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



  Average training loss: 0.65
  Training epcoh took: 0:00:20

Running Validation...


C:\Users\DeepLearning_1\anaconda3\envs\windup\lib\site-packages\ipykernel_launcher.py:107: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


  Accuracy: 0.74
  Validation took: 0:00:00

======== Epoch 2 / 4 ========
Training...

  Average training loss: 0.53
  Training epcoh took: 0:00:19

Running Validation...
  Accuracy: 0.80
  Validation took: 0:00:00

======== Epoch 3 / 4 ========
Training...

  Average training loss: 0.39
  Training epcoh took: 0:00:19

Running Validation...
  Accuracy: 0.79
  Validation took: 0:00:00

======== Epoch 4 / 4 ========
Training...

  Average training loss: 0.28
  Training epcoh took: 0:00:19

Running Validation...
  Accuracy: 0.80
  Validation took: 0:00:00

Training complete!


In [41]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(torch.tensor(b_input_ids).to(device).long(),
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))

C:\Users\DeepLearning_1\anaconda3\envs\windup\lib\site-packages\ipykernel_launcher.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


  Batch   100  of    119.    Elapsed: 0:00:03.

Accuracy: 0.79
Test took: 0:00:04


In [42]:
# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 128

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [43]:
# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(torch.tensor(b_input_ids).to(device).long(), 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [49]:
logits = test_sentences(["‘6-1→8-9 역전패’ SK 염경엽 감독 “야구가 생각대로 된다면…”▲ SK 염경엽 감독. ⓒ곽혜미 기자 [스포티비뉴스=사직, 박대성 기자] SK 와이번스 염경엽 감독이 전날 역전패를 아쉬워했다.  SK는 8일 사직구장에서 롯데 자이언츠와 원정 1차전을 치렀다. 6회까지 정진기, 고종욱, 김창평, 정현이 타점을 올리며 6점을 냈지만, 뒷심 부족으로 무너졌다. 8-8 동점으로 맞이한 연장 10회말 김주한의 폭투로 8-9 역전패를 당했다.  다음날인 9일 만난 염경엽 감독은 “초반 분위기는 아주 좋았다. 그러나 야구가 생각대로 되면 다 쉽지 않겠나”라며 아쉬워했다. 선발투수 문승원을 놓고는 “투구수가 적었지만, 첫 경기라 많이 지쳤다. 제구가 위로 몰릴 때 컨트롤 할 수 있는 능력이 있다면 더 잘할 것이다. 투수 코치가 충분히 이야기했으리라고 본다”고 말했다. 이날 문승원은 5.2이닝 동안 9안타, 7삼진, 4실점을 기록하고 내려왔다.  역전패를 했지만, 큰 그림을 안고 올 시즌을 바라보는 염 감독이다. 염 감독은 “성적도 성적이지만, 육성도 중요하다. 어린 선수를 키우는 과정이 있다. 얼마만큼 빨리 채워지는지가 코칭스태프와 내가 해야 할 몫이다”고 말했다.  9일 열릴 2차전은 우천취소가 됐다. 다음날인 10일 경기에서 선발 변화는 없다. 김태훈이 그대로 나온다. 염 감독은 “김태훈은 올해 10승은 할 수 있다고 생각한다. 지난해 통계를 봤을 때 좌투수들이 전체 성적이 좋았다. 좌타자가 많은 것도 있지만, 일단 통계상으로는 그랬다. 김태훈은 충분히 할 수 있을 것이다”고 기대했다.  스포티비뉴스=사직, 박대성 기자  스포티비뉴스가 여러분의 스포츠 현장 제보(jebo@spotvnews.co.kr)를 기다립니다.  [PL] '블루드래곤' 이청용의 볼튼시절 리그 골모음  [스포츠타임] 4대 센터 차례로 격파…하킴 올라주원의 2년 연속 우승    <저작권자 ⓒ SPOTV NEWS 무단전재 및 재배포 금지> "])
print(logits)
print(np.argmax(logits))

[[-1.709144   1.2614201]]
1


C:\Users\DeepLearning_1\anaconda3\envs\windup\lib\site-packages\ipykernel_launcher.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
